In [ ]:
import numpy as np
import scipy
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_probability as tfp
tfd = tfp.distributions

In [ ]:
import optimizers

In [ ]:
def gamma_factory(params):
    return tfp.distributions.Gamma(concentration=params[0], rate=params[1])

def lognormal_factory(params):
    return tfp.distributions.LogNormal(loc=params[0], scale=params[1])

In [ ]:
alpha = 2.
beta = 5.
gamma = tfd.Gamma(concentration=alpha, rate=beta)

def grad_log_like(x):
    with tf.GradientTape() as g:
        tf_x = tf.constant(x, dtype=tf.float32)
        g.watch(tf_x)
        return g.gradient(gamma.log_prob(tf_x), tf_x).numpy()

grad_log_like(np.array([0.3,0.2]))

In [ ]:
class TFContinuousParameterModel:
    def __init__(self, q_factory, initial_params, params_count, sample_count):
        assert initial_params.ndim == 1
        self.q_factory = q_factory
        #self.param_matrix = np.full((params_count, len(initial_params)), initial_params)
        self.param_matrix = np.copy(initial_params)
        self.sample_count = sample_count
        # The current stored sample.
        self.x = None
        # The gradient of x with respect to the parameters of q.
        self.grad_x = None
        # The stochastic gradient of log sum q for x.
        self.grad_log_sum_q = None
        
    
    def sample(self):
        with tf.GradientTape(persistent=True) as g:
            tf_params = tf.constant(self.param_matrix, dtype=tf.float32)
            g.watch(tf_params)
            q_distribution = self.q_factory(tf_params)
            tf_x = q_distribution.sample(self.sample_count)
            q_term = tf.math.reduce_sum(tf.math.log(q_distribution.prob(tf_x)))
        self.x = tf_x.numpy()
        self.grad_x = g.jacobian(tf_x, tf_params).numpy()
        self.grad_log_sum_q = g.gradient(q_term, tf_params).numpy()
        del g  # Should happen anyway but being explicit to remember.
        return self.x
    
    def elbo_gradient(self, grad_log_p_x): 
        assert self.grad_x is not None
        return np.matmul(grad_log_p_x, self.grad_x) - self.grad_log_sum_q

    
m = TFContinuousParameterModel(gamma_factory, np.array([2.,1]), 3, 5)
print(m.__dict__)
x = m.sample()
m.elbo_gradient(grad_log_like(x))